# jpegio 사용법
### (Last updated 2019.03.19)
##### *오류를 발견하시거나, 필요하신 기능이 있는 경우 언제든지 알려주십시오 (leedaewon@nsr.re.kr, daewon4you@gmail.com)*



- `jpegio`는 C언어로 구현된 [libjpeg](https://www.ijg.org)의 일부 JPEG 입출력 기능을 파이썬 모듈로 만들어(i.e., wrapping) API로 제공하는 파이썬 패키지이다.
- 그리스 ITI-CERTH 연구소 [MKLab](https://mklab.iti.gr)에서 제공하는 [image-forensics](https://github.com/MKLab-ITI/image-forensics) 소스코드를 참고하였다.
- Uber research에서도 [비슷한 코드](https://github.com/uber-research/jpeg2dct)를 제공한다.
- C코드로부터 파이썬 모듈을 생성하기 위해 [Cython](https://cython.org/)을 활용하였다.
- Microsoft Windows에서는 [libjpeg-turbo](https://libjpeg-turbo.org)를 이용한다.
- UNIX 계열 운영체제에서는 `jpegio` 패키지 설치과정에서 `libjpeg`의 소스코드를 컴파일하는 과정을 포함한다.




In [1]:
import jpegio as jio

#### JPEG 이미지 읽어들이기
- 기본적으로 압축이 해제된 JPEG 데이터는 `DecompressedJpeg` 개체를 통해 다루게 된다.
- DCT 계수를 별도의 자료구조로 다루기 위해 다른 개체(e.g., `ZigzagDct1d`)를 이용할 수도 있다 (뒤에서 자세히 설명).

In [2]:
fpath = "../tests/images/cherries01.jpg"  # JPEG 파일 주소
jpeg = jio.read(fpath)
type(jpeg)

jpegio.decompressedjpeg.DecompressedJpeg

#### 이미지 크기 확인

In [3]:
jpeg.image_width, jpeg.image_width

(756, 756)

#### YCbCr 채널(channel) 개수 확인

In [4]:
jpeg.num_components

3

#### DCT 계수(coefficients) 접근
- 멤버 변수명은 `coef_arrays` 이며, 파이썬의 기본 리스트(list) 개체이다.
- `coef_arrays` 리스트는 각 채널에 해당되는 DCT 계수 배열들을 담고 있다.
- 각 DCT 계수 배열은 2차원 `numpy.ndarray` 개체이다.
- DCT 계수 배열을 3차원 `numpy.ndarray`로 관리하지 않는 이유는 채널에 따라 DCT 계수 배열의 크기가 다를 수 있기 때문이다.
- 채널에 따라 DCT 계수 배열의 크기가 다른 경우는, JPEG 압축 과정에서 CbCr 채널에 대해 down sampling이 적용된 경우이다.

In [5]:
type(jpeg.coef_arrays) 

list

In [6]:
type(jpeg.coef_arrays[0])

numpy.ndarray

In [7]:
len(jpeg.coef_arrays)  # 채널의 개수와 동일

3

In [8]:
print(jpeg.coef_arrays[0].ndim)  # 첫번째 DCT 계수 배열의 차원
print(jpeg.coef_arrays[1].ndim)  # 두번째 DCT 계수 배열의 차원
print(jpeg.coef_arrays[2].ndim)  # 세번째 DCT 계수 배열의 차원

2
2
2


In [9]:
# 각 채널별 첫번째 DCT 계수 블록만 출력
for i in range(jpeg.num_components):
    coef = jpeg.coef_arrays[i]
    print("[Channel #%d] The 1st DCT coef. block"%(i+1))
    print(coef[:8, :8])

[Channel #1] The 1st DCT coef. block
[[-567   11  -47   -6    4    2    1    1]
 [ -81  -41   22   13   -5   -2   -2   -4]
 [  19   15   10  -12    3   -1    1    1]
 [   8   -7   -7   -4    3   -3    0   -2]
 [  -7   -5    6    3   -2   -2   -2   -2]
 [  10    0  -10   -5   -3    1   -1   -1]
 [  -2   -4   -1    2    2   -2   -2    1]
 [  -3    0    4   -3   -3    0   -1   -1]]
[Channel #2] The 1st DCT coef. block
[[-58   6   1   8   0   0   2   0]
 [ 11   5  -4   3   6  -3   1  -2]
 [  5  -7  -2   1  -2  -2  -2   0]
 [ -3  -3   5  -1  -3   1   2   2]
 [  3   0  -6  -2   2   1  -1   0]
 [  4  -1   2   1  -1  -3  -2  -1]
 [ -2   0   2   0   0  -1   0   0]
 [  2   0  -2  -1   1   2   1  -1]]
[Channel #3] The 1st DCT coef. block
[[-4 -4 -3 -2  0  1  2 -1]
 [-2 -2 -3  3  0  1 -2  0]
 [-9  9  2 -1  3 -1  0  0]
 [ 4  1 -2 -5 -2  0  0  0]
 [-3 -2 -2 -1  0 -1  1  0]
 [-2  1  1 -1  1  1  0  0]
 [ 1 -2 -1 -1  0  1  0 -1]
 [ 0 -2  0  0  0 -1  0  0]]


In [10]:
# 각 채널별 첫번째 DCT 계수 배열의 크기 출력
# (DCT 배열의 크기가 다른 것을 알 수 있다)
for i in range(jpeg.num_components):
    coef = jpeg.coef_arrays[i]
    print("[Channel #%d] Size of DCT coef. array: %s"%(i+1, coef.shape))

[Channel #1] Size of DCT coef. array: (504, 760)
[Channel #2] Size of DCT coef. array: (256, 384)
[Channel #3] Size of DCT coef. array: (256, 384)


#### DCT 계수 `numpy.ndarray` 배열 모양 변형
- DCT 계수 배열을 보다 효율적으로 사용하기 위해서는 배열 모양을 변형할 필요가 있다.
- 예를 들어, 블록 단위로 처리하고자 하는 경우, 인덱스를 (블록 행, 블록 열, 8x8배열 행, 8x8배열 열)과 같이 사용하는 것이 더욱 용이하다.
- `numpy.reshape`과 `numpy.transpose`를 적절히 사용한다.
- `numpy.reshape`과 `numpy.transpose`는 내부 메모리 구조를 변경하지 않고 데이터를 바라보는 관점(view)만 바꾸기 때문에, 성능 문제를 크게 걱정하지 않아도 된다.

In [11]:
# 8x8 블록 단위로 배열에 접근하고자 할 때,
# 아래와 같이 배열 모양을 바꿀 수 있다.

coef = jpeg.coef_arrays[0]  # 첫번째 채널 DCT 계수 배열
nr_blk = coef.shape[0] // 8  # 8x8 블록 단위 행의 개수
nc_blk = coef.shape[1] // 8  # 8x8 블록 단위 열의 개수
print(nr_blk, nc_blk)

63 95


In [12]:
# 8개 단위로 블록 열개수(nc_blk)만큼 자르고,
# 다시 블록 열을 8개씩 블록 행개수(nr_blk) 만큼 자른다.

coef_blk = coef.reshape(nr_blk, 8, nc_blk, 8)
print(coef_blk.shape)

(63, 8, 95, 8)


In [13]:
# 인덱싱을 사용하기 편하도록 배열의 축(axis) 위치를 바꾼다.
# 데이터가 내부적으로 어떻게 저장되는지 고려할 필요 없이,
# 인덱싱만 원하는 대로 된다고 생각하면 된다 (어차피 내부 데이터 메모리는 1차원 배열).

coef_blk = coef_blk.transpose(0, 2, 1, 3)
print(coef_blk.shape)

(63, 95, 8, 8)


In [14]:
# 3행2열에 있는 DCT 계수 블록
coef_blk[3, 2, :, :]

array([[-644,   -6,    3,  -11,    2,   -2,    7,   -1],
       [ -26,  -27,   28,    5,   -5,    0,   -4,    0],
       [  -4,   -6,    7,   15,   -1,   -2,   -4,    3],
       [   2,   17,  -17,    0,   -1,    1,    1,    0],
       [   0,   -8,    5,    0,    4,   -2,   -2,    1],
       [  -5,   -8,    6,    4,   -1,    0,   -3,   -2],
       [   1,    2,   -3,   -2,   -1,    0,    3,    0],
       [   2,    1,    4,   -1,    0,   -2,   -1,   -1]], dtype=int16)

In [15]:
# 10행10열에 있는 DCT 계수 블록
coef_blk[10, 10, :, :]

array([[-674,    0,  -51,   -3,   16,   -3,    0,   -3],
       [ -61,   15,  -58,   14,   15,   10,   -4,    2],
       [  -4,  -11,  -13,    2,   15,    4,    3,    1],
       [   8,   -2,  -13,   -3,    5,   -2,   -2,    1],
       [   2,    2,   -2,   -7,    3,   -5,   -2,    0],
       [ -16,    6,    8,    1,   -7,   -5,    3,    0],
       [  -2,   -1,    8,    1,   -9,   -4,    0,    1],
       [   4,    0,   -5,   -3,    5,   -1,    5,    0]], dtype=int16)

In [16]:
# 아래와 같이 블록 인덱스만 넣어줘도 된다.
coef_blk[3, 4]

array([[-680,  -29,    7,   -8,   -2,   -3,   -1,    4],
       [ -19,  -28,    6,   12,   -4,    3,    0,    2],
       [  -5,  -14,    0,    9,    1,   -2,   -1,    1],
       [   0,    8,    6,   -5,   -1,   -4,    0,    4],
       [  -2,   -6,    1,    3,   -5,    0,    0,   -2],
       [  -8,   -5,    0,    5,    6,   -3,    2,    0],
       [   1,   -1,    0,    1,   -1,    0,    1,    2],
       [   0,    3,    1,   -3,   -3,   -1,    0,    0]], dtype=int16)

#### 채널별 정보 확인
- JPEG 채널별 정보를 확인하기 위해서는 `DecompressedJpeg`의 `comp_info`를 멤버 변수를 이용한다.
- Downsampling에 관련된 각종 정보를 담고 있어, 특히 CbCr채널에 downsampling이 적용된 JPEG을 다룰 때 크기 정보를 확인하기 좋다.
- 예를 들어,`ComponentInfo` 개체의 `v_samp_factor` 및 `h_samp_factor`는 YCbCr의 각 채널별 downsampling 비율이다. 단순히 downsampling 된 후의 이미지 크기가 필요하다면 `ComponentInfo` 개체의 `downsampled_width`와 `downsampled_height`를 이용하면 된다.

In [17]:
# comp_info는 리스트 개체이며, 각 채널별에 대응되는 ComponentInfo 개체를 담고 있다.
# "component"가 "channel"에 대응된다고 보면 된다.

type(jpeg.comp_info)

list

In [18]:
type(jpeg.comp_info[0])  # ComponentInfo 개체

jpegio.componentinfo.ComponentInfo

In [19]:
jpeg.comp_info[0]  # 첫번째 채널의 ComponentInfo

In [20]:
for ci in jpeg.comp_info:
    print("[Component #%d]"%(ci.component_id))
    print("Quantization table number:", ci.quant_tbl_no)
    print("DC table number:", ci.dc_tbl_no)
    print("AC table number:", ci.ac_tbl_no)
    print("Width after downsampling:", ci.downsampled_width)  # 다운샘플링 후 이미지 가로 크기
    print("Height after downsampling:", ci.downsampled_height)  # 다운샘플링 후 이미지 세로 크기
    print("Width in blocks:", ci.width_in_blocks)  # 블록 행개수
    print("Height in blocks:", ci.height_in_blocks)  # 블록 열개수
    print("Vertical sampling factor:", ci.h_samp_factor)  # 행 샘플링 팩터
    print("Horizontal sampling factor:", ci.v_samp_factor)  # 열 샘플링 팩터
    print()

[Component #1]
Quantization table number: 0
DC table number: 0
AC table number: 0
Width after downsampling: 756
Height after downsampling: 504
Width in blocks: 95
Height in blocks: 63
Vertical sampling factor: 2
Horizontal sampling factor: 2

[Component #2]
Quantization table number: 1
DC table number: 1
AC table number: 1
Width after downsampling: 378
Height after downsampling: 252
Width in blocks: 48
Height in blocks: 32
Vertical sampling factor: 1
Horizontal sampling factor: 1

[Component #3]
Quantization table number: 1
DC table number: 1
AC table number: 1
Width after downsampling: 378
Height after downsampling: 252
Width in blocks: 48
Height in blocks: 32
Vertical sampling factor: 1
Horizontal sampling factor: 1



#### 0이 아닌 DCT AC 계수(non-zero DCT AC coefficient)의 개수  
- 8x8 DCT 계수 블록에서, 가장 첫번째 계수(0행0열)를 DC 계수라고 하며 나머지 계수를 AC 계수라고 한다.
- JPEG의 DCT 계수를 건드리는 대부분의 스테가노그라피 도구들이 AC 계수를 대상으로 하기 때문에 DC 계수를 제외한 AC 계수의 개수를 구할 필요가 있다.
- `jpegio`에서는 `count_nnz_ac`라는 멤버함수를 제공한다. `count_nnz_ac`는 모든 DCT 계수 블록에서, 0이 아닌 AC 계수의 개수를 알려준다. 즉, DC 계수를 제외하고 나머지 계수들 중에서 0이 아닌 계수의 개수를 구한다.

In [21]:
jpeg.count_nnz_ac()

476659

- 만약에 각 채널별로 0이 아닌 AC 계수의 개수를 구하고 싶다면 아래 코드를 이용하면 된다.

In [22]:
import numpy as np

for i in range(jpeg.num_components):
    coef = jpeg.coef_arrays[i]
    nnz_total = np.count_nonzero(coef)  # 모든 DCT 계수 중 0이 아닌 계수의 개수
    nnz_dc = np.count_nonzero(coef[::8, ::8])  # 0이 아닌 DC 계수의 개수
    print("[Channel #%d] Number of non-zero DCT AC coefficients: %d"%(i+1, nnz_total - nnz_dc))

[Channel #1] Number of non-zero DCT AC coefficients: 327921
[Channel #2] Number of non-zero DCT AC coefficients: 76925
[Channel #3] Number of non-zero DCT AC coefficients: 71813


#### Zig-Zag 스캐닝을 통해 DCT 계수를 1차원 배열로 읽어오기
- 필요에 따라 zig-zag 스캐닝 방식으로 읽어들인 DCT 계수의 1차원 배열이 필요할 수 있다.
- 파이썬에서 블록 단위로 zig-zag 스캐닝 처리를 하게되면 성능이 다소 저하될 수 있다.
- `jpegio`에서는 `DecompressedJpeg`의 서브클래스인 `ZigzagDct1d` 클래스를 제공한다.
- JPEG을 `ZigzagDct1d` 개체로 읽어오기 위해서 플래그(flag)를 지정해주어야 한다.

In [23]:
# 참고로, DecompressedJpeg은 jpegio.DECOMPRESSED로 지정돼 있다.
jpeg_zz = jio.read(fpath, jio.ZIGZAG_DCT_1D)

In [24]:
type(jpeg_zz)

jpegio.zigzagdctjpeg.ZigzagDct1d

In [25]:
coef = jpeg_zz.coef_arrays[0]
coef.shape

(63, 95, 64)

- DCT 계수 배열의 마지막 차원의 크기가 64(8x8배열의 2차원이 아닌 1차원 배열 크기)인 것을 알 수 있다.
- 다음은 파이썬 코드와 zig-zag 스캐닝 성능을 비교한 결과이다.

In [26]:
import os
import glob
import time

BS = 8  # Size of the DCT square block width

list_fpaths = []

for fpath in glob.glob(os.path.join('../tests/images', '*.jpg')):
    list_fpaths.append(fpath)
        
for fpath in list_fpaths:
        # Read DCT with ZigzagDct1d
        time_beg_zz = time.time()
        jpeg_zz = jio.read(fpath, jio.ZIGZAG_DCT_1D)
        list_coef_zz = []
        for c in range(jpeg_zz.num_components):                
            nrows_blk, ncols_blk = jpeg_zz.get_coef_block_array_shape(c)

            arr_zz = jpeg_zz.coef_arrays[c].reshape(nrows_blk*ncols_blk, BS*BS)
            list_coef_zz.append(arr_zz)
        # end of for
        time_elapsed_zz = time.time() - time_beg_zz

        # Read DCT with DecompressedJpeg
        time_beg_de = time.time()
        jpeg_de = jio.read(fpath, jio.DECOMPRESSED)
        list_coef_de = []
        for c in range(jpeg_de.num_components):
            arr_de = jpeg_de.coef_arrays[c]
            nrows_blk, ncols_blk = jpeg_de.get_coef_block_array_shape(c)
            arr_de = arr_de.reshape(nrows_blk, BS, ncols_blk, BS)
            arr_de = arr_de.transpose(0, 2, 1, 3)
            arr_de = arr_de.reshape(nrows_blk, ncols_blk, BS, BS)

            zz_de = np.zeros((nrows_blk, ncols_blk, BS*BS),
                             dtype=np.int16)

            # Zigzag scanning over DCT blocks.
            for i in range(nrows_blk):
                for j in range(ncols_blk):
                    zz_de[i, j][0] = arr_de[i, j][0, 0]

                    zz_de[i, j][1] = arr_de[i, j][0, 1]
                    zz_de[i, j][2] = arr_de[i, j][1, 0]

                    zz_de[i, j][3] = arr_de[i, j][2, 0]
                    zz_de[i, j][4] = arr_de[i, j][1, 1]
                    zz_de[i, j][5] = arr_de[i, j][0, 2]

                    zz_de[i, j][6] = arr_de[i, j][0, 3]
                    zz_de[i, j][7] = arr_de[i, j][1, 2]
                    zz_de[i, j][8] = arr_de[i, j][2, 1]
                    zz_de[i, j][9] = arr_de[i, j][3, 0]

                    zz_de[i, j][10] = arr_de[i, j][4, 0]
                    zz_de[i, j][11] = arr_de[i, j][3, 1]
                    zz_de[i, j][12] = arr_de[i, j][2, 2]
                    zz_de[i, j][13] = arr_de[i, j][1, 3]
                    zz_de[i, j][14] = arr_de[i, j][0, 4]

                    zz_de[i, j][15] = arr_de[i, j][0, 5]
                    zz_de[i, j][16] = arr_de[i, j][1, 4]
                    zz_de[i, j][17] = arr_de[i, j][2, 3]
                    zz_de[i, j][18] = arr_de[i, j][3, 2]
                    zz_de[i, j][19] = arr_de[i, j][4, 1]
                    zz_de[i, j][20] = arr_de[i, j][5, 0]

                    zz_de[i, j][21] = arr_de[i, j][6, 0]
                    zz_de[i, j][22] = arr_de[i, j][5, 1]
                    zz_de[i, j][23] = arr_de[i, j][4, 2]
                    zz_de[i, j][24] = arr_de[i, j][3, 3]
                    zz_de[i, j][25] = arr_de[i, j][2, 4]
                    zz_de[i, j][26] = arr_de[i, j][1, 5]
                    zz_de[i, j][27] = arr_de[i, j][0, 6]

                    zz_de[i, j][28] = arr_de[i, j][0, 7]
                    zz_de[i, j][29] = arr_de[i, j][1, 6]
                    zz_de[i, j][30] = arr_de[i, j][2, 5]
                    zz_de[i, j][31] = arr_de[i, j][3, 4]
                    zz_de[i, j][32] = arr_de[i, j][4, 3]
                    zz_de[i, j][33] = arr_de[i, j][5, 2]
                    zz_de[i, j][34] = arr_de[i, j][6, 1]
                    zz_de[i, j][35] = arr_de[i, j][7, 0]

                    zz_de[i, j][36] = arr_de[i, j][7, 1]
                    zz_de[i, j][37] = arr_de[i, j][6, 2]
                    zz_de[i, j][38] = arr_de[i, j][5, 3]
                    zz_de[i, j][39] = arr_de[i, j][4, 4]
                    zz_de[i, j][40] = arr_de[i, j][3, 5]
                    zz_de[i, j][41] = arr_de[i, j][2, 6]
                    zz_de[i, j][42] = arr_de[i, j][1, 7]

                    zz_de[i, j][43] = arr_de[i, j][2, 7]
                    zz_de[i, j][44] = arr_de[i, j][3, 6]
                    zz_de[i, j][45] = arr_de[i, j][4, 5]
                    zz_de[i, j][46] = arr_de[i, j][5, 4]
                    zz_de[i, j][47] = arr_de[i, j][6, 3]
                    zz_de[i, j][48] = arr_de[i, j][7, 2]

                    zz_de[i, j][49] = arr_de[i, j][7, 3]
                    zz_de[i, j][50] = arr_de[i, j][6, 4]
                    zz_de[i, j][51] = arr_de[i, j][5, 5]
                    zz_de[i, j][52] = arr_de[i, j][4, 6]
                    zz_de[i, j][53] = arr_de[i, j][3, 7]

                    zz_de[i, j][54] = arr_de[i, j][4, 7]
                    zz_de[i, j][55] = arr_de[i, j][5, 6]
                    zz_de[i, j][56] = arr_de[i, j][6, 5]
                    zz_de[i, j][57] = arr_de[i, j][7, 4]

                    zz_de[i, j][58] = arr_de[i, j][7, 5]
                    zz_de[i, j][59] = arr_de[i, j][6, 6]
                    zz_de[i, j][60] = arr_de[i, j][5, 7]

                    zz_de[i, j][61] = arr_de[i, j][6, 7]
                    zz_de[i, j][62] = arr_de[i, j][7, 6]

                    zz_de[i, j][63] = arr_de[i, j][7, 7]
                # end of for (j)
            # end of for (i)
            list_coef_de.append(zz_de)
        # end of for (c)
        time_elapsed_de = time.time() - time_beg_de
        print("[File: %s]"%(os.path.basename(fpath)))
        print("[Time] C-optimized: %f, Naive Python: %f" \
              %(time_elapsed_zz, time_elapsed_de), end='\n\n')

[File: arborgreens01.jpg]
[Time] C-optimized: 0.008000, Naive Python: 0.441206

[File: arborgreens02.jpg]
[Time] C-optimized: 0.010014, Naive Python: 0.406220

[File: arborgreens03.jpg]
[Time] C-optimized: 0.012001, Naive Python: 0.442000

[File: arborgreens04.jpg]
[Time] C-optimized: 0.009000, Naive Python: 0.423001

[File: arborgreens05.jpg]
[Time] C-optimized: 0.009999, Naive Python: 0.453000

[File: arborgreens06.jpg]
[Time] C-optimized: 0.011000, Naive Python: 0.484000

[File: arborgreens07.jpg]
[Time] C-optimized: 0.009002, Naive Python: 0.483999

[File: arborgreens08.jpg]
[Time] C-optimized: 0.010000, Naive Python: 0.477998

[File: arborgreens09.jpg]
[Time] C-optimized: 0.010000, Naive Python: 0.483999

[File: arborgreens10.jpg]
[Time] C-optimized: 0.009001, Naive Python: 0.490000

[File: cherries01.jpg]
[Time] C-optimized: 0.008000, Naive Python: 0.468999

[File: cherries02.jpg]
[Time] C-optimized: 0.011000, Naive Python: 0.476000

[File: cherries03.jpg]
[Time] C-optimized: 0.0